In [3]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
import openpyxl
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

# Navigators

In [4]:
''' NAVIGATOR BETWAY '''
bet_way = 'https://betway.bet.ar/es-ar/sports/lve/soccer'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#bway = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bway = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bway.get(bet_way)
bway.maximize_window()

In [5]:
''' NAVIGATOR 1XBET '''
url_xbet = 'https://ar.1xbet.com/es/live/football'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator
options.add_argument('window-size=1920x1080') #Headless = True
#xbet = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
xbet = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
xbet.get(url_xbet)
xbet.set_window_size(1920, 1080)

In [6]:
''' NAVIGATOR BETSSON '''
url_bet = 'https://pba.betsson.bet.ar/apuestas-deportivas/futbol'

#changing chromedriver default options
options = Options()
options.headless = False #for not opening the navigator

#bets = webdriver.Chrome(r'C:\Users\Willy\Desktop\Ale\UpWork\LIVE-ODDS-Scrape-and-calculate-different-strategies\chromedriver.exe', options=options)
bets = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bets.get(url_bet)
bets.maximize_window()

In [7]:
''' NAVIGATOR       20BET LIVE '''
url = 'https://20bet.com/es'
options = Options()
options.headless = False #for not opening the navigator

veinte = webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
veinte.get(url)
veinte.maximize_window()

notific = WebDriverWait(veinte, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.align-right.secondary.slidedown-button')))
notific.click()

cookie = WebDriverWait(veinte, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="platform"]/div[2]/div[3]/div/button')))
cookie.click()


In [8]:
''' NAVIGATOR BODOG ''' 

url_bod = 'https://www.bodog.com/es/sports/futbol'

options = Options()

bod =  webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bod.get(url_bod)
bod.maximize_window()

# Functions

In [48]:
def betway():
    df_bway = pd.DataFrame(columns =['teams', 'home', 'tie', 'away'])
    df_bway2 = pd.DataFrame(columns=['teams', 'over', 'goals', 'under', 'site'])
    for j in range(30):
        for i in range(5):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                
                players_bway = elemento[3]
                home = elemento[7]
                tie  = elemento[8]
                away = elemento[9]

                df_bway.loc[len(df_bway.index)] = [players_bway, home, tie, away]
                #print(players_bway, '|         |', f'j: {j}, i: {i} ')

            except:
                pass
    time.sleep(0.5)
    dropdown = bway.find_element(By.XPATH, '/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div[2]/div/div[1]')
    dropdown.click()
    time.sleep(1.5)
    TotalGoals = bway.find_element(By.CSS_SELECTOR, '[collectionitem="total-goals"]')
    TotalGoals.click()
    
    time.sleep(1.5)
    for j in range(20):
        for i in range(10):
            try:
                elemento = bway.find_element(By.XPATH, f'/html/body/div[1]/div/div[3]/div/div[1]/div/div[2]/div[4]/div/div[2]/div/div[2]/div[1]/div/div[{j}]/div[2]/div/div[{i}]').text.split('\n')
                teams = elemento[3]
                goals = elemento[7]
                under = elemento[10]
                over  = elemento[8]
                df_bway2.loc[len(df_bway2.index)] = [teams, over, goals, under, 'betWay']
                #print(teams, f'||     goals: {goals}, under: {under}, over: {over} ')
            except:
                pass

    df_bway = pd.merge(df_bway, df_bway2, on='teams')
    dropdown.click()
    time.sleep(1.5)
    HomeXaway = bway.find_element(By.CSS_SELECTOR, 'div[collectionitem="win-draw-win"]')
    HomeXaway.click()
    df_bway.to_csv('CSV\df_bway.csv')
    return df_bway

In [10]:
df_betway = betway()
df_betway.head()

,teams,home,tie,away,over,goals,under,site
0,Godoy Cruz -Atlético Patronato,"12,00","4,00","1,30","1,61","1,5","2,20",betWay
1,Botafogo -Internacional,"17,00","4,33","1,25","1,80","1,5","1,90",betWay
2,Juventude -Atlético GO,"15,00","4,00","1,28","2,00","1,5","1,72",betWay
3,Athletico PR -Coritiba,"1,77","2,90","5,25","1,72","1,5","2,00",betWay
4,Avai -Fluminense,"9,00","4,50","1,30","1,70","2,5","2,05",betWay


In [49]:
def One_xbet():
    df_xbet = pd.DataFrame(columns=['teams','home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    for k in range(40):
        for j in range(12):
            try:
                elemento =xbet.find_element(By.XPATH, f'/html/body/div[3]/div[1]/div[2]/div/div/div[4]/div/div/div[1]/div/div[{k}]/div[{j}]').text.split('\n')
                teams = elemento[0]+' '+elemento[1]
                if len(elemento)==24:
                    home = elemento[9]
                    tie  = elemento[10]
                    away = elemento[11]

                    over = elemento[15].replace('-', '1')
                    goals = elemento[16].replace('-', '1')
                    under = elemento[17].replace('-', '1')
                    df_xbet.loc[len(df_xbet.index)]= [teams, home, tie, away, over, goals, under]
                if len(elemento)==26:
                    home = elemento[11]
                    tie  = elemento[12]
                    away = elemento[13]

                    over = elemento[17].replace('-', '1')
                    goals = elemento[18].replace('-', '1')
                    under = elemento[19].replace('-', '1')
                    df_xbet.loc[len(df_xbet.index)]= [teams, home, tie, away, over, goals, under, '1XBET']

                
                
                #print(elemento[0], elemento[1],'         |', k,'  ', j, '|',len(elemento))
            except:
                pass
    #df_xbet = df_xbet[~df_xbet.teams.str.contains('Campeonato')]
    df_xbet.to_csv('CSV\df_xbet.csv')
    return df_xbet

In [12]:
df_xbet = One_xbet()
df_xbet

,teams,home,tie,away,over,goals,under,site
0,FC Godoy Cruz Patronato (Paraná),16,4.62,1.33,1.67,1.5,2.34,1XBET
1,CA Independiente Avellaneda FC Banfield,1.06,9.5,51,1.77,1.5,2.17,1XBET
2,Botafogo (Río de Janeiro) Internacional (Porto...,23,4.42,1.3,1.93,1.5,1.95,1XBET
3,Esporte Clube Juventude Atletico Clube Goianiense,26,4.13,1.32,1.97,1.5,1.91,1XBET
4,Peñarol (Montevideo) River Plate de Montevideo,5.5,3.25,1.73,2.27,2.5,1.64,1XBET
5,Unión Magdalena Deportivo (Pereira),1.44,3.85,12.3,1.63,1.5,2.39,1XBET
6,Celaya Correcaminos UAT,29,21,1.01,1.6,3.5,2.34,1XBET
7,Colorado Springs Mountain Lions Colorado Schoo...,-,-,-,1,1,1,1XBET
8,Humboldt State Lumberjacks Cal State San Marco...,17,10,1.001,1.86,2.5,1.944,1XBET
9,Chico State Wildcats CSUSB Coyotes,4.05,1.952,3.275,1.56,0.5,2.43,1XBET


In [50]:
def betsson_live():    
    df_bet = pd.DataFrame(columns=['teams','home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    bool=False
    for k in range(30):
        try:
            elemento = bets.find_element(By.CSS_SELECTOR, f'obg-event-row-container[test-id^="f-"]:nth-of-type({k})').text.split('\n')
            #print(elemento[1]+' '+ elemento[2], f'|| len:{len(elemento)}, nth-of-type: {k}')
            
            teams = elemento[1]+' '+elemento[2]
            
            home = elemento[9]
            tie = elemento[11]
            away = elemento[13]
            
            for q in range(len(elemento)):
                if 'Ganador Del Partido' in elemento[q]:
                    home = elemento[q+3]
                    tie  = elemento[q+5]
                    away = elemento[q+7]
                if 'más de ' in elemento[q]:
                    over = elemento[q+1]
                    goals = str(float("".join(re.findall(r'\d+', elemento[q+2])))/10)
                    under = elemento[q+3]
                    #print(over, goals, under)
                    break
            
            while bool:
                if len(elemento)==23:
                    under = elemento[19]
                    goals = float("".join(re.findall(r'\d+', elemento[18])))/10
                    over  = elemento[17]
                
                if len(elemento)==30:
                    under = elemento[18]
                    goals = float("".join(re.findall(r'\d+', elemento[17])))/10
                    over = elemento[16]

                if len(elemento)==35:
                    under = elemento[19]
                    goals = str(float("".join(re.findall(r'\d+', elemento[18])))/10)
                    over = elemento[17]

                if len(elemento) == 21:
                    under = elemento[18]
                    goals = float("".join(re.findall(r'\d+', elemento[17])))/10
                    over = elemento[16]

            df_bet.loc[len(df_bet.index)]  = [teams, home.replace('-','1'), tie.replace('-','1'), away.replace('-','1'), over.replace('-','1'), goals.replace('-','1'), under.replace('-','1'), 'BetSSon']
        except:
            pass
    df_bet = df_bet[df_bet.away != 'CERRAR APUESTA']
    columns = ['home', 'tie', 'away', 'over', 'goals', 'under']
    #for col in columns:
    #    df_bet[col]=df_bet[col].astype('float64')
    df_bet.to_csv('CSV\df_bet_live.csv')
    return df_bet

In [23]:
df_betsson = betsson_live()
df_betsson

Sport Vasco da Gama || len:23, nth-of-type: 1
Universitario FBC Melgar || len:19, nth-of-type: 2
Botafogo Internacional || len:23, nth-of-type: 3
Juventude Atlético-GO || len:20, nth-of-type: 4
Godoy Cruz Patronato Paraná || len:23, nth-of-type: 5
Independiente Banfield || len:23, nth-of-type: 6
Municipal (GUA) Antigua GFC || len:23, nth-of-type: 7
Unión Magdalena Deportivo Pereira || len:23, nth-of-type: 8
Almagro Independiente Rivadavia || len:23, nth-of-type: 9
Estudiantes Río Cuarto Deportivo Riestra || len:23, nth-of-type: 10
Real Santa Cruz Always Ready || len:23, nth-of-type: 11
Peñarol River Plate || len:23, nth-of-type: 12
Athletico-PR Coritiba || len:35, nth-of-type: 13
Avaí Fluminense || len:35, nth-of-type: 14
Liverpool (ECF_RuBIX) Tottenham Hotspur (ECF_Bomb1to) || len:18, nth-of-type: 15
Manchester City (ECF_DangerDim77) Manchester United (ECF_JKey) || len:18, nth-of-type: 16


,teams,home,tie,away,over,goals,under,site
0,Sport Vasco da Gama,6.00,1.22,11.00,4.00,2.5,1.25,BetSSon
1,Universitario FBC Melgar,4.75,1.22,34.00,4.00,2.5,1.25,BetSSon
2,Botafogo Internacional,26.00,4.75,1.22,2.09,1.5,1.68,BetSSon
3,Juventude Atlético-GO,4.75,1.42,6.50,2.09,1.5,1.68,BetSSon
4,Godoy Cruz Patronato Paraná,26.00,4.75,1.22,2.09,1.5,1.68,BetSSon
5,Independiente Banfield,1.06,9.50,67.00,2.11,1.5,1.66,BetSSon
6,Municipal (GUA) Antigua GFC,3.40,1.57,5.50,2.00,0.5,1.70,BetSSon
7,Unión Magdalena Deportivo Pereira,1.33,3.75,15.00,1.76,1.5,1.95,BetSSon
8,Almagro Independiente Rivadavia,12.00,3.50,1.36,1.65,3.5,2.06,BetSSon
9,Estudiantes Río Cuarto Deportivo Riestra,1.22,4.33,17.00,1.69,1.5,1.99,BetSSon


## ---------------------------------------------------------------------------------------

In [80]:
def bet20_live():
    df_20bet = pd.DataFrame(columns=['teams','home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    elementos = veinte.find_elements(By.CSS_SELECTOR, f'div[class="event-table__row"]')
    i=0
    for partido in elementos:
        elemento = partido.text.split('\n')
        partido.location_once_scrolled_into_view
        #time.sleep(0.15)
        if len(elemento)==17:
            teams = elemento[2]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 1
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 1')
        if len(elemento)==18:
            teams = elemento[2]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[7]
            tie   = elemento[8]
            away  = elemento[9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 2
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 2')
        if len(elemento)==19:
            teams = elemento[2]+' '+elemento[5]+' '+elemento[4]
            teams = ''.join([i for i in teams if not i.isdigit()])
            home  = elemento[-11]
            tie   = elemento[-10]
            away  = elemento[-9]
            under = elemento[-3]
            goals = elemento[-2]
            over  = elemento[-1]
            tipo  = 3
            #print(teams,'|', home, tie, away,'||' ,under, goals, over, f' len: {len(elemento)}, i: {i}, tipo 3')
        for a in range(len(elemento)):
            try:
                if int(elemento[a])>100:
                    home = elemento[a+1]
                    tie  = elemento[a+2]
                    away = elemento[a+3]
            except:
                pass
        try:

            df_20bet.loc[len(df_20bet.index)] = [teams, home, tie, away, over, goals, under, '20BET']
        except:
            pass
            

        

        
    elementos[0].location_once_scrolled_into_view
    df_20bet.drop_duplicates('teams',inplace=True)
    df_20bet.to_csv('CSV\df_20bet_live.csv')
    return df_20bet

In [75]:
for a in range(len(elemento)):
    try:
        if int(elemento[a])>100:
            print(elemento[a+1], elemento[a+2], elemento[a+3])
            
            
    except:
        pass

1.9 2.55 6.5


In [81]:
df_bet20 = bet20_live()
df_bet20

,teams,home,tie,away,over,goals,under,site
0,Millonarios Patriotas FC,1.4,4.8,7.25,1.9,2.5,1.8,20BET
1,Peñarol CA River Plate (Uru),3.9,1.57,5.4,2.1,2.5,1.7,20BET
2,Barcelona SC Aucas,2.8,3,2.65,1.7,2,2.05,20BET
3,Mushuc Runa SC Independiente Del Valle,7.75,4.2,1.42,1.65,2,2.15,20BET
4,Universidad Católica CS Emelec,1.65,3.7,5.2,1.7,2,2.1,20BET
5,CS Cienciano Alianza Lima,1.85,3.2,4,1.65,2,2.1,20BET
6,Portuguesa Deportivo Lara,2.65,2.8,2.75,1.9,2,1.8,20BET


# -----------------------------------------------------------------------------

In [131]:
def bodog_live():
    df_bodog = pd.DataFrame(columns=['teams', 'home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    for j in range(10):
        for k in range(5):
            try:
                elemento = bod.find_element(By.XPATH, f'/html/body/bx-site/ng-component/div/sp-sports-ui/div/main/div/section/main/sp-path-event/div/div[2]/sp-happening-now/div/div/div/div[{j}]/sp-coupon[{k}]/sp-multi-markets/section/article').text.split('\n')
                if len(elemento)>10:
                    teams = elemento[2]+' '+elemento[3]+' '+elemento[4]+' '+elemento[5]+' '+elemento[6]+' '+elemento[7]
                    teams = ''.join([i for i in teams if not i.isdigit()]).replace('+  Apuestas', '')
                    numbers = elemento[-5]+' '+elemento[-4]+' '+elemento[-3]+' '+elemento[-2]+' '+elemento[-1]
                    numbers = numbers.split(' ')

                    home=numbers[0].replace(',', '')
                    tie = numbers[1].replace(',', '')
                    away = numbers[2].replace(',', '')
                    if len(numbers)==7:
                        goals = numbers[3].replace(',', '').replace(',', '')
                        over = numbers[4].replace("(", "").replace(")","").replace(',', '')
                        under = numbers[6].replace("(", "").replace(")","").replace(',', '')
                    else:
                        goals = 1
                        over  = 1
                        under = 1


                    df_bodog.loc[len(df_bodog.index)]=[teams, home, tie, away, over, goals, under, 'bodog']
                    #print(teams,'|', numbers,  f'|| {len(elemento)}', 'j',j,'k', k)
                    #print(numbers)
            except:
                pass
    df_bodog.to_csv('CSV\df_bodog.csv')
    return df_bodog

In [132]:
df_bod = bodog_live()
df_bod

,teams,home,tie,away,over,goals,under,site
0,Athletico Paranaense Coritiba,1.01,14.00,451.00,5.25,0.5,1.16,bodog
1,Millonarios Boyaca Patriotas,1.629,2.90,8.00,1,1,1,bodog
2,Cienciano Alianza Lima,2.20,2.38,4.90,1,1,1,bodog
3,Barcelona SC Aucas,3.50,2.14,3.05,1.80,1.0,1.952,bodog
4,Mushuc Runa Independiente del Valle,8.00,2.72,1.699,1,1,1,bodog
5,Universidad Católica Emelec,1.317,4.30,11.00,1,1,1,bodog
6,Club Nacional Club Guaraní,1.167,5.60,23.00,1,1,1,bodog
7,Peñarol River Plate Montevideo,12.50,1.04,14.00,4.30,2.5,1.20,bodog
8,Unión San Felipe Puerto Montt,1.40,3.60,8.50,2.65,2.5,1.444,bodog


# ----------------------------------------------------------------------------

In [52]:
def merge(margen=55):
    lista = [   ['df_xbet', 'df_bodog'],
                ['df_bway', 'df_bodog'],
                ['df_bet_live', 'df_bodog'],
                ['df_20bet_live', 'df_bodog'],
                ['df_bway','df_xbet' ],
                ['df_xbet', 'df_bet_live'],
                ['df_xbet', 'df_20bet_live'],
                ['df_bway', 'df_20bet_live'],
                ['df_bway', 'df_bet_live'],
                ['df_bet_live', 'df_20bet_live']]
    
    lista2 = [['df_bodog', 'df_xbet', 'df_20bet_live']]

    df_main = pd.DataFrame(columns=['teams_x', 'home_x', 'tie_x', 'away_x', 'over_x', 'goals_x', 'under_x','site_x', 'teams_matched', 'score', 'teams_y', 'home_y', 'tie_y','away_y', 'over_y', 'goals_y', 'under_y', 'site_y'])
    
    for i in range(len(lista)):
        df1 = pd.read_csv(f'CSV\{lista[i][0]}.csv')
        df2 = pd.read_csv(f'CSV\{lista[i][1]}.csv')

        teams_list = df2.teams.tolist()

        df1[['teams_matched', 'score']] = df1.teams.apply(lambda x: process.extractOne(x, teams_list, scorer=fuzz.token_sort_ratio)).apply(pd.Series) 
        df1 = df1[df1.score>margen]

        df3 = pd.merge(df1, df2, left_on='teams_matched', right_on='teams', how='left')
        df3.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
        df_main=pd.concat([df_main, df3], axis=0)


        
        #print(f'df_new: {df3.shape}')
        #print(f'df_main: {df_main.shape}')
    columns = [ 'home_x', 'tie_x', 'away_x', 'over_x', 'goals_x', 'under_x',
            'home_y', 'tie_y', 'away_y', 'over_y', 'goals_y', 'under_y',]
    df_main = df_main[df_main.goals_x == df_main.goals_y]
    print(f'Filtered to: {df_main.shape} ')
    df_main.to_excel('CSV\df_main_excel.xlsx')
    for col in columns:
        df_main[col] = df_main[col].astype(str).str.replace('.', ',')
    df_main.to_csv('CSV\df_main.csv')
    df_main.columns

    return df_main

In [41]:
merge()

Filtered to: (33, 18) 


,teams_x,home_x,tie_x,away_x,over_x,goals_x,under_x,site_x,teams_matched,score,teams_y,home_y,tie_y,away_y,over_y,goals_y,under_y,site_y
0,FC Godoy Cruz Patronato (Paraná),16,"4,62","1,33","1,67","1,5","2,34",1XBET,Godoy Cruz Patronato,82,Godoy Cruz Patronato,"51,0","5,75","1,125","3,2","1,5","1,364",bodog
1,CA Independiente Avellaneda FC Banfield,"1,06","9,5",51,"1,77","1,5","2,17",1XBET,Independiente Banfield,72,Independiente Banfield,"1,056","8,5","101,0","2,88","1,5","1,435",bodog
2,Botafogo (Río de Janeiro) Internacional (Porto...,23,"4,42","1,3","1,93","1,5","1,95",1XBET,Botafogo Internacional,62,Botafogo Internacional,"36,0","5,2","1,16","2,95","1,5","1,408",bodog
5,Unión Magdalena Deportivo (Pereira),"1,44","3,85","12,3","1,63","1,5","2,39",1XBET,Unión Magdalena Deportes Pereira,92,Unión Magdalena Deportes Pereira,"1,328","3,7","14,0","2,28","1,5","1,588",bodog
0,Universitario FBC Melgar,"4,75","1,22","34,0","4,0","2,5","1,25",BetSSon,Universitario de Deportes FBC Melgar,80,Universitario de Deportes FBC Melgar,"8,75","1,029","41,0","3,6","2,5","1,26",bodog
1,Botafogo Internacional,"26,0","4,75","1,22","2,09","1,5","1,68",BetSSon,Botafogo Internacional,100,Botafogo Internacional,"36,0","5,2","1,16","2,95","1,5","1,408",bodog
3,Godoy Cruz Patronato Paraná,"26,0","4,75","1,22","2,09","1,5","1,68",BetSSon,Godoy Cruz Patronato,87,Godoy Cruz Patronato,"51,0","5,75","1,125","3,2","1,5","1,364",bodog
4,Independiente Banfield,"1,06","9,5","67,0","2,11","1,5","1,66",BetSSon,Independiente Banfield,100,Independiente Banfield,"1,056","8,5","101,0","2,88","1,5","1,435",bodog
5,Unión Magdalena Deportivo Pereira,"1,33","3,75","15,0","1,76","1,5","1,95",BetSSon,Unión Magdalena Deportes Pereira,92,Unión Magdalena Deportes Pereira,"1,328","3,7","14,0","2,28","1,5","1,588",bodog
0,Godoy Cruz Patronato Paraná,"55,0","5,8","1,12","2,75","1,5","1,4",20BET,Godoy Cruz Patronato,87,Godoy Cruz Patronato,"51,0","5,75","1,125","3,2","1,5","1,364",bodog


In [42]:
def merge_3way(margen = 60):

    df_3way = pd.DataFrame(columns=['teams_x', 'home_x', 'tie_x', 'away_x', 'over_x', 'goals_x', 'under_x',
                                        'site_x', 'teams_matched', 'score', 'teams_y', 'home_y', 'tie_y',
                                        'away_y', 'over_y', 'goals_y', 'under_y', 'site_y', 'teams', 'home',
                                        'tie', 'away', 'over', 'goals', 'under', 'site'])

    combinaciones = [['df_bodog', 'df_xbet', 'df_20bet_live'],['df_bway' , 'df_xbet', 'df_20bet_live'],
                    ['df_bet_live', 'df_xbet', 'df_20bet_live'],['df_bway' , 'df_xbet', 'df_20bet_live'],
                    ['df_bet_live', 'df_bodog', 'df_20bet_live'],['df_bet_live', 'df_bway', 'df_20bet_live'],
                    ['df_bway', 'df_bodog', 'df_xbet'],['df_bet_live','df_bodog', 'df_xbet'],
                    ['df_bet_live', 'df_bway', 'df_xbet'],['df_bet_live', 'df_bway', 'df_bodog']]
    ''' juntar los 2 primeros elementos de la lista''' 
    for i in range(len(combinaciones)):
        try:
            df1 = pd.read_csv(f'CSV\{combinaciones[i][0]}.csv')
            df2 = pd.read_csv(f'CSV\{combinaciones[i][1]}.csv')

            teams_list = df2.teams.tolist()

            df1[['teams_matched', 'score']] = df1.teams.apply(lambda x: process.extractOne(x, teams_list, scorer=fuzz.token_sort_ratio)).apply(pd.Series) 
            df1 = df1[df1.score>margen]

            df3 = pd.merge(df1, df2, left_on='teams_matched', right_on='teams', how='left')
            df3.drop(columns=['Unnamed: 0_x','Unnamed: 0_y'], inplace=True)
            #print(f'2Merged shape: {df3.shape} ')
            df4 = pd.read_csv(f'CSV\{combinaciones[i][2]}.csv')
            teams_list2 = df4.teams.tolist()
            
            df3[['teams_matched', 'score']] = df3.teams_x.apply(lambda x:process.extractOne(x, teams_list2, scorer=fuzz.token_sort_ratio)).apply(pd.Series)
            df3 = df3[df3.score>margen]

            df5 = pd.merge(df3, df4, left_on='teams_matched', right_on='teams', how='left')
            df5.drop(columns=['Unnamed: 0'], inplace=True)
            #print(df5.columns)
            df_3way = pd.concat([df_3way, df5], axis=0)
            #print(f'{combinaciones[i]}',df_3way.shape)
        except:
            #print(f'Trouble with: {combinaciones[i]} ')
            pass
    columns = ['home_x', 'tie_x', 'away_x', 'over_x', 'goals_x', 'under_x', 'home_y', 'tie_y',
    'away_y', 'over_y', 'goals_y', 'under_y', 'home',
    'tie', 'away', 'over', 'goals', 'under']


    
    for col in columns:
        df_3way[col] = df_3way[col].astype(str).str.replace('.', ',')
    df_3way.to_csv('CSV\df_3way.csv')
    return df_3way

In [43]:
merge_3way()

,teams_x,home_x,tie_x,away_x,over_x,goals_x,under_x,site_x,teams_matched,score,...,under_y,site_y,teams,home,tie,away,over,goals,under,site
0,Botafogo Internacional,"36,0","5,2","1,16","2,95","1,5","1,408",bodog,Botafogo FR RJ Internacional RS,83,...,"1,95",1XBET,Botafogo FR RJ Internacional RS,"35,0","4,9","1,18","2,55","1,5","1,45",20BET
1,Juventude Atlético Goianiense,"6,0","1,23","8,0","4,0","2,5","1,25",bodog,EC Juventude RS Atlético Goianiense,90,...,"1,91",1XBET,EC Juventude RS Atlético Goianiense,"1,25","8,75","1,05","3,55","2,5","1,25",20BET
2,Godoy Cruz Patronato,"51,0","5,75","1,125","3,2","1,5","1,364",bodog,Godoy Cruz Patronato Paraná,87,...,"2,34",1XBET,Godoy Cruz Patronato Paraná,"55,0","5,8","1,12","2,75","1,5","1,4",20BET
3,Independiente Banfield,"1,056","8,5","101,0","2,88","1,5","1,435",bodog,CA Independiente CA Banfield,88,...,"2,17",1XBET,CA Independiente CA Banfield,"1,05","8,0","100,0","2,55","1,5","1,45",20BET
4,Unión Magdalena Deportes Pereira,"1,328","3,7","14,0","2,28","1,5","1,588",bodog,Unión Magdalena Santa Marta Deportivo Pereira,77,...,"2,39",1XBET,Unión Magdalena Santa Marta Deportivo Pereira,"1,32","3,75","16,5","2,2","1,5","1,65",20BET
5,Peñarol River Plate Montevideo,"5,75","3,25","1,658","2,5","2,0","2,0",bodog,Peñarol CA River Plate (Uru),70,...,"1,64",1XBET,Peñarol CA River Plate (Uru),"3,3","1,65","2,15","1,6","2,0","2,2",20BET
0,Godoy Cruz -Atlético Patronato,"12,00","4,00","1,30","1,61","1,5","2,20",betWay,Godoy Cruz Patronato Paraná,74,...,"2,34",1XBET,Godoy Cruz Patronato Paraná,"55,0","5,8","1,12","2,75","1,5","1,4",20BET
1,Botafogo -Internacional,"17,00","4,33","1,25","1,80","1,5","1,90",betWay,Botafogo FR RJ Internacional RS,83,...,"1,95",1XBET,Botafogo FR RJ Internacional RS,"35,0","4,9","1,18","2,55","1,5","1,45",20BET
0,Botafogo Internacional,"26,0","4,75","1,22","2,09","1,5","1,68",BetSSon,Botafogo FR RJ Internacional RS,83,...,"1,95",1XBET,Botafogo FR RJ Internacional RS,"35,0","4,9","1,18","2,55","1,5","1,45",20BET
1,Godoy Cruz Patronato Paraná,"26,0","4,75","1,22","2,09","1,5","1,68",BetSSon,Godoy Cruz Patronato Paraná,100,...,"2,34",1XBET,Godoy Cruz Patronato Paraná,"55,0","5,8","1,12","2,75","1,5","1,4",20BET


# ---------------------------

In [46]:
def final_function(minutos):
    # TIME STUFF
    target = pd.datetime.now() + pd.to_timedelta(minutos, unit='min')
    target_minute = target.minute
    target_hour = target.hour
    now = pd.datetime.now()

    i=0
    #THREADS
    # Put everything to work
    while target>now:

        
        betway_thread  = threading.Thread(target=betway)
        One_xbet_thread   = threading.Thread(target=One_xbet)
        betsson  = threading.Thread(target=betsson_live)
        bet20 = threading.Thread(target=bet20_live)
        bodog = threading.Thread(target=bodog_live)
        merge1 = threading.Thread(target=merge)
        merge2 = threading.Thread(target=merge_3way)
        
        clear_output()
        i+=1
        print(f'Will work till: {target_hour}hs and {target_minute}mins')
        print(f'Iteration number: {i}')
        print('-------')
        print('Functions running')

        betway_thread.start()
        One_xbet_thread.start()
        betsson.start()
        bet20.start()
        bodog.start()

        betway_thread.join()
        One_xbet_thread.join()
        betsson.join()
        bet20.join()
        bodog.join()
        print('Scrapers done')
        print('-------')
        

        print('Merging start...')
        merge1.start()
        merge2.start()
        merge1.join()
        merge2.join()
        print('-------')
        print('Merging finished! All csv files have been updated')
        time.sleep(5)
        now = pd.datetime.now()
    

In [130]:
final_function(10)

Will work till: 20hs and 59mins
Iteration number: 35
-------
Functions running
  Athletico Paranaense  Coritiba  | ['1.01', '14.00', '451.00', '0.5', '(5.25)', '0.5', '(1.16)'] || 15 j 1 k 1
['1.01', '14.00', '451.00', '0.5', '(5.25)', '0.5', '(1.16)']
  Millonarios  Boyaca Patriotas  | ['1.629', '2.93', '8.00', '1,', '1.5', '(1.80)', '1,', '1.5', '(1.952)'] || 14 j 2 k 1
['1.629', '2.93', '8.00', '1,', '1.5', '(1.80)', '1,', '1.5', '(1.952)']
  Cienciano  Alianza Lima  | ['2.20', '2.38', '4.90', '1,', '1.5', '(2.05)', '1,', '1.5', '(1.741)'] || 13 j 3 k 1
['2.20', '2.38', '4.90', '1,', '1.5', '(2.05)', '1,', '1.5', '(1.741)']
  Barcelona SC   Aucas | ['3.50', '2.14', '3.05', '1.0', '(1.80)', '1.0', '(1.952)'] || 16 j 4 k 1
['3.50', '2.14', '3.05', '1.0', '(1.80)', '1.0', '(1.952)']
  Mushuc Runa Independiente del Valle   | ['8.00', '2.74', '1.69', '1,', '1.5', '(1.909)', '1,', '1.5', '(1.833)'] || 13 j 4 k 2
['8.00', '2.74', '1.69', '1,', '1.5', '(1.909)', '1,', '1.5', '(1.833)']
  Un

KeyboardInterrupt: 